In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
from functions.spotipyTools import *
from dotenv import load_dotenv
import os
import openpyxl

In [ ]:
load_dotenv()

In [ ]:
e_client_id = os.environ["client_id"]
e_client_key = os.environ["client_key"]
e_playlist_id = os.environ["playlist_id"]
e_user_id = os.environ["user_id"]

sp_login = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=e_client_id,
                                               client_secret=e_client_key,
                                               redirect_uri="http://localhost:8888/callback", # Need to whitelist the URL
                                               scope="user-read-recently-played"))

In [ ]:
sp_login.current_user_recently_played()